## Setup Telemetry

In [14]:
import os
import dotenv

dotenv.load_dotenv()

from phoenix.otel import register

OTEL_EXPORTER_OTLP_HEADERS = os.getenv("OTEL_EXPORTER_OTLP_HEADERS")
PHOENIX_CLIENT_HEADERS = os.getenv("PHOENIX_CLIENT_HEADERS")
PHOENIX_COLLECTOR_ENDPOINT = os.getenv("PHOENIX_COLLECTOR_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OTEL_EXPORTER_OTLP_HEADERS or not PHOENIX_CLIENT_HEADERS or not PHOENIX_COLLECTOR_ENDPOINT or not OPENAI_API_KEY:
    raise ValueError("Missing required environment variables")

endpoint = PHOENIX_COLLECTOR_ENDPOINT + "/v1/traces"


In [15]:
project_name = "donors_game"

tracer_provider = register(
    project_name=project_name,
    endpoint=endpoint
)

from openinference.semconv.resource import ResourceAttributes
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes
from openinference.semconv.trace import OpenInferenceSpanKindValues


resource = Resource(attributes={
  ResourceAttributes.PROJECT_NAME: project_name
})

tracer_provider = TracerProvider(resource=resource)
trace.set_tracer_provider(tracer_provider=tracer_provider)
tracer = trace.get_tracer(__name__)
span_exporter = OTLPSpanExporter(endpoint=endpoint)
simple_span_processor = SimpleSpanProcessor(span_exporter=span_exporter)
trace.get_tracer_provider().add_span_processor(simple_span_processor)


Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


OpenTelemetry Tracing Details
|  Phoenix Project: donors_game
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [23]:
from openinference.instrumentation.openai import OpenAIInstrumentor
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

import openai
client = openai.OpenAI(api_key=OPENAI_API_KEY)


Attempting to instrument while already instrumented


## Configurations, Typings, and Player Class

In [77]:
from pydantic import BaseModel, Field
from typing import Any, List, Optional
from time import sleep
import json


class GameState(BaseModel):
  generation: int
  round: int
  
  # configs
  donation_multiplier: float = 2
  trace_depth: int = 3
  base_wallet: int = 10
  generations: int = 10
  rounds: int = 12
  players: int = 12
  cutoff_threshold: float = 0.5




class Decision(BaseModel):
  recipient_snapshot: Any
  game_state_snapshot: GameState
  donation_percent: float
  
  class Config:
    arbitrary_types_allowed = True

class StrategyBuilder(BaseModel):
  """Build the strategy"""
  thoughts: List[str] = Field(..., description="Briefly describe your thought process for the strategy to take for this round.")
  strategy: str = Field(..., description="The strategy to be used, must begin with 'My strategy will be'.")

class DonationBuilder(BaseModel):
  """Build the donation"""
  thoughts: List[str] = Field(..., description="Briefly describe your thought process for the donation to make for this round.")
  donation: float = Field(..., description="The percentage amount of resources to donate. MUST BE A FLOAT BETWEEN 0 AND 1.")

def structured_generation_wrapper(*args, **kwargs) -> dict:
  sleep(.1)
  span = trace.get_current_span()
  args_to_log = {**kwargs, "response_format": kwargs["response_format"].model_json_schema()}
  span.set_attribute(SpanAttributes.INPUT_VALUE, json.dumps(args_to_log))
  res = client.beta.chat.completions.parse(**kwargs)
  span.set_attribute(SpanAttributes.OUTPUT_VALUE, res.model_dump_json())
  so = res.choices[0].message.parsed
  return so

class Player:
  def __init__(self, game_state: GameState, i: int, parents: List["Player"] = [], strategy: Optional[str] = None):
    self.name: str = f"{game_state.generation}_{i}"
    self.parents: List[Player] = parents
    self.history: List[Decision] = [] 
    self.wallet: int = game_state.base_wallet
    if strategy:
      self.strategy: str = strategy
    else:
      self.strategy: str = self.generate_strategy(game_state=game_state, parents=parents)
    

  
  def clone(self):
    player = Player(name=self.name, parents=self.parents, strategy=self.strategy)
    player.wallet = self.wallet
    player.history = self.history
    return player



  def system_prompt(self, game_state: GameState):
    return f"""Each player is given an initial endowment of 10 units of a resource.
In each round, you are randomly paired with another individual.
One of you is a donor, and the other is a recipient.
The donor decides to give up some amount of the resource.
The recipient receives {game_state.donation_multiplier}x the number of units that the donor gave up.
If you were a donor in one round, you will be a recipient in the next round, and vice versa.
Your goal is to maximize the number of units you have after the final round.
After the game has finished, the best-performing half of agents will survive to the next generation, and continue playing."""

  def strategy_prompt(self, parents: List["Player"] = []):
    if parents: 
      parent_advice = "\n".join([f"\n\n{parent.name} with score {parent.wallet} strategy: {parent.strategy}" for parent in parents])
      generation_prompt = f"""How would you approach the game? Here is the advice of the best-performing 50% of the previous generation, along with their final scores.
{parent_advice}
Modify this advice to create your own strategy."""
    else: 
      generation_prompt = "Based on the description of the game, create a strategy that you will follow in  the game."
    
    return f"""Our name is {self.name}.

{generation_prompt}

As a donor, you will receive limited information about the recipient and their recent interactions: You will see what the recipient (let’s call them A) did in the immediately preceding round only.
You will also see what A’s partner from that round (let’s call them B) did in their preceding round.
This chain continues for at most three rounds back.
Important: You do NOT have access to multiple rounds of information about A directly.
You only know what A did in the single most recent round.

Example (assuming it’s now round 4): You are paired with A.
You know: In round 3, A donated X% to B.
You also know: In round 2, B donated Y% to C.
And finally: In round 1, C donated Z% to D.
Remember: This is all the information you have.
You don’t know what A did in rounds 1 or 2, only in round 3.

In the first round, there will be no information about the recipient’s previous behavior - think carefully about how much to donate in that case.
Before formulating your strategy, briefly think step by step about what would be a successful strategy in this game.
Then describe your strategy briefly without explanation in one sentence that starts: My strategy will be."""
  
  def donation_prompt(self, game_state: GameState, recipient: "Player"):
    def build_trace(depth: int):
      trace = ""
      current_recipient = recipient
      for _ in range(depth):
        if not current_recipient.history:
          break
        trace += f"In round {current_recipient.history[0].game_state_snapshot.round}, {current_recipient.name} donated {current_recipient.history[0].donation_percent}% of their resources to {current_recipient.history[0].recipient_snapshot.name}.\n"
        current_recipient = current_recipient.history[0].recipient_snapshot
      return trace
    
    trace = build_trace(game_state.trace_depth)

    # TODO: donor is invisible to recipient
    return f"""Your name is 2_4.
As you will recall, here is the strategy you decided to follow: {self.strategy}
This is generation {game_state.generation}.
It is now round {game_state.round}.
In this round, you have been paired with {recipient.name}.
They currently have {recipient.wallet} units of the valuable resource.
Here is what {recipient.name} and their earlier partners did in previous rounds: {trace}
You will take the role of donor. You currently have {self.wallet} units of the valuable resource.

How many units do you give up? Very briefly think step by step about how you apply your strategy in this situation and then provide your answer."""
  
  def generate_strategy(self, game_state: GameState, parents: List["Player"] = []) -> str:
    with tracer.start_as_current_span("generate_strategy") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.LLM.value)
      messages = [
        {"role": "system", "content": self.system_prompt(game_state=game_state)},
        {"role": "user", "content": self.strategy_prompt(parents=parents)},
      ]
      args = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "response_format": StrategyBuilder
      }
      res = structured_generation_wrapper(**args)
      built_strategy = StrategyBuilder.model_validate(res)

      print(f"\n\n{self.name} strategy:\n{built_strategy.model_dump_json(indent=2)}")

      return built_strategy.strategy
  
  def generate_donation(self, game_state: GameState, recipient: "Player") -> float:
    with tracer.start_as_current_span("generate_donation") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.LLM.value)
      messages=[
        {"role": "system", "content": self.system_prompt(game_state=game_state)},
        {"role": "user", "content": self.donation_prompt(game_state=game_state, recipient=recipient)},
      ]
      args = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "response_format": DonationBuilder
      }
      res = structured_generation_wrapper(**args)
      built_donation = DonationBuilder.model_validate(res)

      print(f"\n\n{self.name} donation:\n{built_donation.model_dump_json(indent=2)}")

      return built_donation.donation
  
  def execute_donation(self, recipient: "Player", game_state: GameState):
    with tracer.start_as_current_span("execute_donation") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.AGENT.value)
      span.set_attribute(SpanAttributes.INPUT_VALUE, json.dumps(self.model_dump()))
      donation_percent = self.generate_donation(game_state, recipient)
      donation_amount = self.wallet * donation_percent
      self.wallet -= donation_amount
      recipient.wallet += donation_amount * game_state.donation_multiplier
      span.set_attribute(SpanAttributes.OUTPUT_VALUE, json.dumps({
        "donation_percent": donation_percent,
        "donation_amount": donation_amount,
      }))

      game_state_copy = GameState(**game_state.model_dump())
      recipient_copy = Player(game_state=game_state_copy, i = recipient.name.split("_")[1], parents=recipient.parents, strategy=recipient.strategy)
      decision = Decision(recipient_snapshot=recipient_copy, game_state_snapshot=game_state_copy, donation_percent=donation_percent)

      self.history.append(decision)
      
      return donation_percent
  
  # get current user information as a dict
  def model_dump(self):
    return {
      "name": self.name,
      "parents": [parent.name for parent in self.parents],
      "history": [{
        "recipient_snapshot": decision.recipient_snapshot.model_dump(),
        "game_state_snapshot": decision.game_state_snapshot.model_dump(),
        "donation_percent": decision.donation_percent
      } for decision in self.history],
      "wallet": self.wallet,
      "strategy": self.strategy
    }
  
  # function to make the player json serializable
  def __json__(self):
    return self.model_dump()

### Test Player

In [73]:
# test by creating two players
game_state = GameState(generation=1, round=1)
player1 = Player(game_state=game_state, i=1)
player2 = Player(game_state=game_state, i=2)



1_1 strategy:
{
  "thoughts": [
    "In the first round, I have no information about the recipient, so I will start with a conservative donation to test their willingness to reciprocate in future rounds.",
    "In subsequent rounds, I will analyze the recipient's previous donation behavior to inform my own decisions: if they donated a significant portion, I will reciprocate generously; if they did not, I will either donate less or nothing at all.",
    "I will aim for a balanced strategy that encourages cooperation while also protecting my own resource endowment."
  ],
  "strategy": "My strategy will be to start by donating 2 units in the first round, then adjust my donations based on the recipient's previous behavior in subsequent rounds."
}


1_2 strategy:
{
  "thoughts": [
    "In the first round, there is no prior information available so I will donate a small amount to establish trust and observe how the other player reacts in the next round.",
    "In subsequent rounds, I will 

In [53]:
player1.execute_donation(player2, game_state)

Invalid type dict for attribute 'donation' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




1_1 donation:
{
  "thoughts": [
    "I start with an initial strategy of donating 1 unit to test the waters.",
    "I only have 9 units, so I cannot donate more than that.",
    "1_2 has more units than I do, indicating they might be exhibiting a more generous or cooperative strategy.",
    "I'll stick to my original plan of donating 1 unit to keep some resource for myself while still contributing positively."
  ],
  "donation": 0.1111111111111111
}


0.1111111111111111

In [50]:
print(player1.wallet)
print(player2.wallet)

8.0
14.0


## Orchestrator

In [78]:
from typing import Tuple
from typing import List
import numpy as np

class Orchestrator:
  def __init__(self, game_state: GameState):
    with tracer.start_as_current_span("init_orchestrator") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.CHAIN.value)
      span.set_attribute(SpanAttributes.INPUT_VALUE, game_state.model_json_schema())
      self.game_state = game_state
      self.players = [Player(game_state=game_state, i=i) for i in range(game_state.players)]
  
  def create_donor_recipient_pairs(self) -> List[Tuple[Player, Player]]:
    # each player should be both a donor and recipient and it should be randomized 
    pairs: List[Player] = []
    donors_map = {}
    recipient_pool: List[Player] = self.players.copy()
    donor_pool: List[Player] = self.players.copy()
    for donor in donor_pool:
      recipient = np.random.choice(recipient_pool)
      while recipient == donor or donors_map.get(recipient.name, "") == donor.name:
        recipient = np.random.choice(recipient_pool)
      pairs.append((donor, recipient))
      recipient_pool.remove(recipient)
    
    return pairs
  
  def play_round(self):
    with tracer.start_as_current_span("play_round") as span:
      span.set_attribute(SpanAttributes.INPUT_VALUE, self.game_state.model_dump_json())
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.CHAIN.value)
      pairs = self.create_donor_recipient_pairs()
      span.set_attribute(SpanAttributes.OUTPUT_VALUE, json.dumps([f"{donor.name} -> {recipient.name}" for donor, recipient in pairs]))
      for donor, recipient in pairs:
        donor.execute_donation(recipient, self.game_state)
      self.game_state.round += 1

  def evolve(self):
    with tracer.start_as_current_span("evolve") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.AGENT.value)
      span.set_attribute(SpanAttributes.INPUT_VALUE, self.game_state.model_dump_json())
      self.game_state.round = 1
      self.game_state.generation += 1
      print(f"\n\nEvolving to generation {self.game_state.generation}")
      # sort players by wallet
      self.players = sorted(self.players, key=lambda x: x.wallet, reverse=True)
      # get top half
      top_half = self.players[:int(len(self.players)*self.game_state.cutoff_threshold)]
      # clone players
      top_players_strings = "\n".join([f"player {player.name} with wallet {player.wallet} and strategy: {player.strategy}" for player in top_half])
      print(f"\n\nTop half:\n{top_players_strings}")
      self.players = [Player(game_state=self.game_state, i=i, parents=top_half) for i in range(self.game_state.players)]

  def run(self) -> List[Player]:
    for generation_count in range(self.game_state.generations):
      with tracer.start_as_current_span(f"generation_{generation_count}") as span:
        span.set_attribute(SpanAttributes.INPUT_VALUE, self.game_state.model_dump_json())
        span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.CHAIN.value)
        print(f"\n\nGeneration {self.game_state.generation}")
        for round_count in range(self.game_state.rounds):
          with tracer.start_as_current_span(f"round_{round_count}") as span:
            span.set_attribute(SpanAttributes.INPUT_VALUE, self.game_state.model_dump_json())
            span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.CHAIN.value)
            print(f"\n\n\tGeneration {self.game_state.generation} Round {self.game_state.round}")
            self.play_round()
        if self.game_state.generation < self.game_state.generations:
          self.evolve()
    
    return self.players
    
  


    



## Run Orchestrator

In [80]:
from opentelemetry import trace

with tracer.start_as_current_span("donors_game") as span:
  try:
    # game_state = GameState(generation=1, round=1, generations=4, rounds=6, players=4, cutoff_threshold=0.5)
    game_state = GameState(generation=1, round=1, generations=2, rounds=3, players=2, cutoff_threshold=0.5)
    span.set_attribute(SpanAttributes.INPUT_VALUE, game_state.model_dump_json())
    span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.CHAIN.value)
    orchestrator = Orchestrator(game_state)
    final_players = orchestrator.run()
  except Exception as e:
    span.set_status(trace.Status(trace.StatusCode.ERROR))
    span.record_exception(e)

final_players = sorted(final_players, key=lambda x: x.wallet, reverse=True)
top_players_strings = "\n".join([f"player {player.name} with wallet {player.wallet} and strategy: {player.strategy}" for player in final_players])
print(f"\n\nFinal players: \n{top_players_strings}")

Invalid type dict for attribute 'input.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




1_0 strategy:
{
  "thoughts": [
    "In the first round, since there's no information on the recipient's past behavior, I will donate a small amount to encourage reciprocation without risking too much of my resource.",
    "From the second round onwards, I will analyze the recipient's most recent donation to their partner to gauge their tendency to be generous or selfish.",
    "If the recipient was generous in their previous round, I will reciprocate with a larger donation, as they are likely to do the same for me in the next round.",
    "If the recipient was selfish, I will reduce my donation significantly to preserve my resources and avoid being taken advantage of."
  ],
  "strategy": "My strategy will be to donate a small amount in the first round and adjust my future donations based on the generosity of my recipient's most recent action."
}


1_1 strategy:
{
  "thoughts": [
    "In the first round, since there is no prior information, I should start by donating a moderate amoun

In [81]:
!pip freeze > requirements.txt